In [34]:
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
import pandas as pd

In [2]:
config = {'max_new_tokens': 256, 'temperature': 0.1}

In [4]:
# Calling llama model
llm = CTransformers(model="D:\Code Workspace\DL Model\llama-2-7b-chat.ggmlv3.q8_0.bin",
                    model_type = 'llama',
                    config = config)

d:\Code Workspace\Personal Project\Personal Project - LangChain Project\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
## Create LLM Chaining
questions_template = "Generate a {selected_topic_level} math quiz on the topic of {selected_topic}. Include {num_quizzes} questions"
questions_prompt = PromptTemplate(input_variables=["selected_topic_level", "selected_topic", "num_quizzes"],
                                    template=questions_template)
questions_chain = LLMChain(llm= llm,
                            prompt = questions_prompt,
                            output_key = "questions")

In [30]:
answer_template = "From this Question:\n {questions}\n\n gave me answer to each one of them"
answer_prompt = PromptTemplate(input_variables = ["questions"],
                                template = answer_template)
answer_chain = LLMChain(llm = llm,
                        prompt = answer_prompt,
                        output_key = "answer")

In [31]:
## Create Sequential Chaining
seq_chain = SequentialChain(chains = [questions_chain, answer_chain],
                            input_variables = ['selected_topic_level', 'selected_topic', 'num_quizzes'],
                            # Here we return multiple variables
                            output_variables = ['questions', 'answer'])

In [32]:
response = {'selected_topic_level': 'Elementary School Level', 
 'selected_topic': 'Basic Arithmetic', 
 'num_quizzes': 2, 
 'questions': ' for each of the following topics: addition, subtraction, multiplication, and division.\n\n1. Addition:\na) 3 + 5 = ?\nb) 7 + 2 = ?\n\n2. Subtraction:\na) 9 - 4 = ?\nb) 10 - 3 = ?\n\n3. Multiplication:\na) 6 x 4 = ?\nb) 8 x 3 = ?\n\n4. Division:\na) 24 / 4 = ?\nb) 50 / 10 = ?\n\nNote: Make sure the questions are challenging but not too difficult for elementary school level students.', 
 'answer': ':\n\n1. Addition:\na) 3 + 5 = 8\nb) 7 + 2 = 9\n\n2. Subtraction:\na) 9 - 4 = 5\nb) 10 - 3 = 7\n\n3. Multiplication:\na) 6 x 4 = 24\nb) 8 x 3 = 24\n\n4. Division:\na) 24 / 4 = 6\nb) 50 / 10 = 5\n\nPlease let me know if you need anything else!'}

In [37]:
response.get('answer')

':\n\n1. Addition:\na) 3 + 5 = 8\nb) 7 + 2 = 9\n\n2. Subtraction:\na) 9 - 4 = 5\nb) 10 - 3 = 7\n\n3. Multiplication:\na) 6 x 4 = 24\nb) 8 x 3 = 24\n\n4. Division:\na) 24 / 4 = 6\nb) 50 / 10 = 5\n\nPlease let me know if you need anything else!'